# 01_IBM-HHRR_ATTRITION_Default_V1

# 0.- Previous Tasks

In [1]:
# Generic Libraries
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  RobustScaler
import numpy as np
import os
from IPython.display import display, HTML  # Importar HTML para aplicar formato

In [2]:
# Define general path:
path_general = r'C:\HHRR'
path_default = os.path.join(path_general,'01_Default') 

In [3]:
# Model Libraries.

# Cross validation
from sklearn.model_selection import cross_val_score 

#------------- / Regresion Logistica /--------------
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

#--------------/ XGBoost /--------------------------
from xgboost import XGBClassifier
import xgboost as xgb

#-------------/ AdaBoost /--------------------------
from sklearn.ensemble import AdaBoostClassifier
#-------------/ CatBoost /--------------------------
from catboost import CatBoostClassifier
#------------/ Decission Tree /----------------------
from sklearn.tree import DecisionTreeClassifier

#------------/ Random Forest /-----------------------
from sklearn.ensemble import RandomForestClassifier

#-----------/   MLP /--------------------------------
from sklearn.neural_network import MLPClassifier

#------------/ KNN /----------------------------------
from sklearn.neighbors import KNeighborsClassifier

#------------/ Naive - Bayes /-------------------------
from sklearn.naive_bayes import GaussianNB


In [4]:
# Metric Libraries
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score,f1_score,fbeta_score, confusion_matrix

# Grid
from sklearn.model_selection import GridSearchCV

In [5]:
# Load dataset
df = pd.read_csv(r'C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\HR-Employee-ML.csv', index_col=0)

In [6]:
df

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_Associate Degree,Education_Bachelor’s Degree,Education_Doctorate,Education_High School,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,annual_salary,economic_impact
0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,11,0,8,0,6,4,0,5,71916,14670.864
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,23,1,10,3,10,7,1,7,61560,12558.240
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,15,0,7,3,0,0,0,0,25080,4037.880
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,11,0,8,3,8,7,3,0,34908,6876.876
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,12,1,6,3,2,2,2,2,41616,8198.352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,17,1,17,3,5,2,0,3,30852,6077.844
1466,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,15,1,9,5,7,7,1,7,119892,25177.320
1467,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,20,1,6,0,6,2,0,3,73704,15035.616
1468,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,14,0,17,3,9,6,0,8,64680,13194.720


In [7]:
X = df.drop(columns='Attrition')
y = df['Attrition']

In [8]:
# Separation of the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1029, 78), (441, 78), (1029,), (441,))

In [9]:
# Check dataset composition
print(" Fraudulent Count for Full data :  ",np.sum(y))
print(" Fraudulent Count for Train data : ",np.sum(y_train))
print(" Fraudulent Count for Test data :   ",np.sum(y_test))

 Fraudulent Count for Full data :   237
 Fraudulent Count for Train data :  166
 Fraudulent Count for Test data :    71


In [10]:
# Save the testing set for evaluation
X_test_saved = X_test.copy()
y_test_saved = y_test.copy()
print("Saved X_test & y_test")

Saved X_test & y_test


# Data Transformation

## Original Dataset 

### Smote

In [11]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_smote))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [12]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_adasyn))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 800})


## Power Transformation

### Original

In [13]:
# - Apply : preprocessing.PowerTransformer(copy=False) to fit & transform the train & test data

from sklearn import metrics 
from sklearn import preprocessing


from sklearn.preprocessing import PowerTransformer

pt= preprocessing.PowerTransformer(method='yeo-johnson', copy=True)  # creates an instance of the PowerTransformer class.
pt.fit(X_train)

X_train_pt = pt.transform(X_train)
X_test_pt = pt.transform(X_test)

y_train_pt = y_train
y_test_pt = y_test

### Smote

In [14]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train_pt))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote_pt, y_train_smote_pt = smote.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_smote_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [15]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn_pt, y_train_adasyn_pt = adasyn.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_adasyn_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 839})


### Model Loading Preparation: Libraries and Functions

In [16]:
# LOAD OF MODELS.
# perfom cross validation on the X_train & y_train 
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold cross-validator
# perform cross validation
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
#  Shuffle is False because we need a constant best model when we use GridSearchCV


In [17]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

In [18]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, f1_score, fbeta_score, confusion_matrix

## Functions

In [19]:
def model_cv(distributions, model_type, save_directory_complete_model=None):
    '''
    This function performs cross-validation and allows training the following algorithms:
        - 'regression_logistic' - 'xgboost' - 'adaboost' - 'catboost'
        - 'decision_tree' - 'random_forest' - 'mlp' - 'knn'
    
    It makes and validates predictions and obtains the most common evaluation metrics.
    
    Parameters:
        - data_list: A list containing the name of the dataset, followed by the train dataset variables [name, X_train, y_train, X_val, y_val].
        - param_values: Primary parameter of the selected algorithm. If not provided, default values are used.
        - model_type: See the list of allowed algorithms; it returns an error if not allowed.
    
    Returns:
        - Returns a dataframe with the most frequent evaluation measures of an algorithm:
            - roc_auc - accuracy - precision - recall - f1_score - f2_score - confusion_matrix 
    '''
    resultados_totales = []
    for distribution in distributions:
        # Unpack the data list
        name = distribution[0]
        X_train, y_train, X_val, y_val = distribution[1:]

        # Create lists to store the results
        roc_auc_scores = []
        accuracy_scores = []
        precision_scores = []
        recall_scores = []
        f1_scores = []
        f2_scores = []
        confusion_matrices = []

        # Create the model according to the specified type
        #-------------/ GRUPO 1 /----------------------------   
        if model_type == 'regression_logistic':
            model = LogisticRegression() 
        #------------/ GRUPO 2 /----------------------------- 
        elif model_type == 'xgboost':
            model = XGBClassifier()
        elif model_type == 'adaboost':
            model = AdaBoostClassifier(DecisionTreeClassifier())
        elif model_type == 'catboost':
            model = CatBoostClassifier(verbose=0)

        #------------/ GRUPO 3 /------------------------------
        elif model_type == 'decision_tree':
            model = DecisionTreeClassifier()
        elif model_type == 'random_forest':
            model = RandomForestClassifier()   
        #------------/ GRUPO 4 /-------------------------------  
        elif model_type == 'mlp':
            model = MLPClassifier()   
        #------------/ GRUPO 5 /-------------------------------    
        elif model_type == 'knn':
            model = KNeighborsClassifier()
        else:
            raise ValueError('Invalid model_type parameter.')

        # Train the model with the training set
        model.fit(X_train, y_train)

        # Get the predictions for the validation set
        y_pred = model.predict(X_val)

        # Calculate the metrics
        roc_auc = roc_auc_score(y_val, y_pred)
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred)
        recall = recall_score(y_val, y_pred)
        f1 = f1_score(y_val, y_pred)
        f2 = fbeta_score(y_val, y_pred, beta=2)
        confusion = confusion_matrix(y_val, y_pred)

        # Add the results to the lists
        roc_auc_scores.append(roc_auc)
        accuracy_scores.append(accuracy)
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        f2_scores.append(f2)
        confusion_matrices.append(confusion)

        # Create the DataFrame with the results
        results_df = pd.DataFrame({
            'Model': [model_type],
            'Description': [name],
            'Parameter': 'Default',
            'ROC-AUC': roc_auc_scores,
            'Accuracy': accuracy_scores,
            'Precision': precision_scores,
            'Recall': recall_scores,
            'F1 Score': f1_scores,
            'F2 Score': f2_scores,
            'Confusion Matrix': confusion_matrices
        })
        resultados_totales.append(results_df)
    df_resultados_final = pd.concat(resultados_totales, ignore_index=True)
    
    # Save current work directory by default
    if save_directory_complete_model is None:
        save_directory_complete_model = os.path.join(os.getcwd(), 'default')
        
    # Create path of file CSV using model name
    save_path = os.path.join(save_directory_complete_model, f"{model_type}_default.csv")
    
    # Create directory if not exits
    os.makedirs(save_directory_complete_model, exist_ok=True)
    
    # Save dataframe as CSV file
    df_resultados_final.to_csv(save_path, index=False)   
    return (df_resultados_final)


In [20]:
def show_models(model_list):
    """
    This function shows models in screen in pretty_paint format
    
    Parameters:
        - data_list: A list containing the name of models.
    """
    for model in model_list:
        display(HTML(f"<h2 style='text-align: center;font-size:30px;'> Modelo {model}:</h2>")) 
    
        # Call function y save result
        my_model = model_cv(distributions, model, path_default)

        # Show DF in screen 
        display(my_model) 
        print("\n\n\n") 

## Create distributions

In [21]:
# Original distribution
OR_origin = ['OR origin',X_train, y_train, X_test, y_test]
OR_smote =['OR smote',X_train_smote, y_train_smote, X_test, y_test]
OR_adasyn = ['OR adasyn', X_train_adasyn, y_train_adasyn, X_test, y_test]

# Power Transformation
PT_origin = ['PT origin',X_train_pt, y_train_pt, X_test_pt, y_test_pt]
PT_smote = ['PT smote',X_train_smote_pt, y_train_smote_pt, X_test_pt, y_test_pt ]
PT_adasyn = ['PT adasyn', X_train_adasyn_pt, y_train_adasyn_pt, X_test_pt, y_test_pt]

# Total Distributions
distributions = [OR_origin, OR_smote, OR_adasyn, PT_origin, PT_smote, PT_adasyn]

In [22]:
model_list = ['regression_logistic', 'adaboost', 'xgboost', 'catboost', 'decision_tree', 'random_forest', 'mlp' , 'knn']

In [23]:
show_models(model_list)

,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,regression_logistic,OR origin,Default,0.504340,0.836735,0.333333,0.014085,0.027027,0.017422,"[[368, 2], [70, 1]]"
1,regression_logistic,OR smote,Default,0.588085,0.662132,0.232877,0.478873,0.313364,0.395349,"[[258, 112], [37, 34]]"
2,regression_logistic,OR adasyn,Default,0.629273,0.664399,0.257862,0.577465,0.356522,0.462754,"[[252, 118], [30, 41]]"
3,regression_logistic,PT origin,Default,0.707785,0.882086,0.711111,0.450704,0.551724,0.486322,"[[357, 13], [39, 32]]"
4,regression_logistic,PT smote,Default,0.740160,0.802721,0.425926,0.647887,0.513966,0.586735,"[[308, 62], [25, 46]]"
5,regression_logistic,PT adasyn,Default,0.740445,0.793651,0.412281,0.661972,0.508108,0.590452,"[[303, 67], [24, 47]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,adaboost,OR origin,Default,0.530529,0.727891,0.204819,0.239437,0.220779,0.231608,"[[304, 66], [54, 17]]"
1,adaboost,OR smote,Default,0.624990,0.800454,0.376812,0.366197,0.371429,0.368272,"[[327, 43], [45, 26]]"
2,adaboost,OR adasyn,Default,0.606072,0.768707,0.313253,0.366197,0.337662,0.354223,"[[313, 57], [45, 26]]"
3,adaboost,PT origin,Default,0.562752,0.743764,0.250000,0.295775,0.270968,0.285326,"[[307, 63], [50, 21]]"
4,adaboost,PT smote,Default,0.623925,0.789116,0.355263,0.380282,0.367347,0.375000,"[[321, 49], [44, 27]]"
5,adaboost,PT adasyn,Default,0.621793,0.766440,0.322222,0.408451,0.360248,0.387701,"[[309, 61], [42, 29]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,xgboost,OR origin,Default,0.613247,0.857143,0.642857,0.253521,0.363636,0.288462,"[[360, 10], [53, 18]]"
1,xgboost,OR smote,Default,0.620575,0.850340,0.571429,0.281690,0.377358,0.313480,"[[355, 15], [51, 20]]"
2,xgboost,OR adasyn,Default,0.621926,0.852608,0.588235,0.281690,0.380952,0.314465,"[[356, 14], [51, 20]]"
3,xgboost,PT origin,Default,0.613247,0.857143,0.642857,0.253521,0.363636,0.288462,"[[360, 10], [53, 18]]"
4,xgboost,PT smote,Default,0.637362,0.859410,0.628571,0.309859,0.415094,0.344828,"[[357, 13], [49, 22]]"
5,xgboost,PT adasyn,Default,0.626266,0.850340,0.567568,0.295775,0.388889,0.327103,"[[354, 16], [50, 21]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,catboost,OR origin,Default,0.590483,0.857143,0.700000,0.197183,0.307692,0.230263,"[[364, 6], [57, 14]]"
1,catboost,OR smote,Default,0.631386,0.868481,0.740741,0.281690,0.408163,0.321543,"[[363, 7], [51, 20]]"
2,catboost,OR adasyn,Default,0.641131,0.875283,0.807692,0.295775,0.432990,0.338710,"[[365, 5], [50, 21]]"
3,catboost,PT origin,Default,0.590483,0.857143,0.700000,0.197183,0.307692,0.230263,"[[364, 6], [57, 14]]"
4,catboost,PT smote,Default,0.645470,0.873016,0.758621,0.309859,0.440000,0.351438,"[[363, 7], [49, 22]]"
5,catboost,PT adasyn,Default,0.644119,0.870748,0.733333,0.309859,0.435644,0.350318,"[[362, 8], [49, 22]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,decision_tree,OR origin,Default,0.561401,0.741497,0.247059,0.295775,0.269231,0.284553,"[[306, 64], [50, 21]]"
1,decision_tree,OR smote,Default,0.593338,0.766440,0.300000,0.338028,0.317881,0.329670,"[[314, 56], [47, 24]]"
2,decision_tree,OR adasyn,Default,0.566235,0.768707,0.275362,0.267606,0.271429,0.269122,"[[320, 50], [52, 19]]"
3,decision_tree,PT origin,Default,0.576551,0.757370,0.275000,0.309859,0.291391,0.302198,"[[312, 58], [49, 22]]"
4,decision_tree,PT smote,Default,0.613894,0.791383,0.352113,0.352113,0.352113,0.352113,"[[324, 46], [46, 25]]"
5,decision_tree,PT adasyn,Default,0.649676,0.784580,0.363636,0.450704,0.402516,0.430108,"[[314, 56], [39, 32]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,random_forest,OR origin,Default,0.544176,0.836735,0.470588,0.112676,0.181818,0.132890,"[[361, 9], [63, 8]]"
1,random_forest,OR smote,Default,0.620289,0.859410,0.655172,0.267606,0.380000,0.303514,"[[360, 10], [52, 19]]"
2,random_forest,OR adasyn,Default,0.611896,0.854875,0.620690,0.253521,0.360000,0.287540,"[[359, 11], [53, 18]]"
3,random_forest,PT origin,Default,0.558260,0.841270,0.526316,0.140845,0.222222,0.165017,"[[361, 9], [61, 10]]"
4,random_forest,PT smote,Default,0.595108,0.845805,0.551724,0.225352,0.320000,0.255591,"[[357, 13], [55, 16]]"
5,random_forest,PT adasyn,Default,0.600799,0.845805,0.548387,0.239437,0.333333,0.269841,"[[356, 14], [54, 17]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,mlp,OR origin,Default,0.507328,0.832200,0.285714,0.028169,0.051282,0.034364,"[[365, 5], [69, 2]]"
1,mlp,OR smote,Default,0.542406,0.757370,0.235294,0.225352,0.230216,0.227273,"[[318, 52], [55, 16]]"
2,mlp,OR adasyn,Default,0.500000,0.160998,0.160998,1.000000,0.277344,0.489655,"[[0, 370], [0, 71]]"
3,mlp,PT origin,Default,0.690997,0.873016,0.666667,0.422535,0.517241,0.455927,"[[355, 15], [41, 30]]"
4,mlp,PT smote,Default,0.701313,0.861678,0.589286,0.464789,0.519685,0.485294,"[[347, 23], [38, 33]]"
5,mlp,PT adasyn,Default,0.702665,0.863946,0.600000,0.464789,0.523810,0.486726,"[[348, 22], [38, 33]]"


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,knn,OR origin,Default,0.505196,0.809524,0.190476,0.056338,0.086957,0.065574,"[[353, 17], [67, 4]]"
1,knn,OR smote,Default,0.506357,0.591837,0.165644,0.380282,0.230769,0.302013,"[[234, 136], [44, 27]]"
2,knn,OR adasyn,Default,0.502589,0.575964,0.162791,0.394366,0.230453,0.307018,"[[226, 144], [43, 28]]"
3,knn,PT origin,Default,0.557975,0.850340,0.692308,0.126761,0.214286,0.151515,"[[366, 4], [62, 9]]"
4,knn,PT smote,Default,0.666483,0.612245,0.257282,0.746479,0.382671,0.540816,"[[217, 153], [18, 53]]"
5,knn,PT adasyn,Default,0.669471,0.607710,0.257143,0.760563,0.384342,0.546559,"[[214, 156], [17, 54]]"
